# 读取数据

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")
train_data = pd.read_csv('train_all.csv',nrows=10000)
test_data = pd.read_csv('test_all.csv',nrows=100)
features_columns = [col for col in train_data.columns if col not in ['user_id','label']]
train = train_data[features_columns].values
test = test_data[features_columns].values
target =train_data['label'].values

# 缺失值补全
处理缺失值有很多方法，最常用为以下几种：
1 填充。（本案例所用方法）通用的方法是采用平均数、中位数来填充，可以适用插值或者模型预测的方法进行缺失补全。
2 删除。当数据量较大时，或者缺失数据占比较小时，可以使用这种方法。
3 不处理。树类模型对缺失值不明感。

In [2]:
# from sklearn.preprocessing import Imputer
# imputer = Imputer(strategy="median")

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(train)
train_imputer = imputer.transform(train)
test_imputer = imputer.transform(test)

# 特征选择
在机器学习和统计学中，特征选择（英语：feature selection）也被称为变量选择、属性选择 或变量子集选择 。它是指：为了构建模型而选择相关特征（即属性、指标）子集的过程。使用特征选择技术有三个原因：

简化模型，使之更易于被研究人员或用户理解，
缩短训练时间，
改善通用性、降低过拟合（即降低方差）。
要使用特征选择技术的关键假设是：训练数据包含许多冗余 或无关 的特征，因而移除这些特征并不会导致丢失信息。 冗余 或无关 特征是两个不同的概念。如果一个特征本身有用，但如果这个特征与另一个有用特征强相关，且那个特征也出现在数据中，那么这个特征可能就变得多余。 特征选择技术与特征提取有所不同。特征提取是从原有特征的功能中创造新的特征，而特征选择则只返回原有特征中的子集。 特征选择技术的常常用于许多特征但样本（即数据点）相对较少的领域。特征选择应用的典型用例包括：解析书面文本和微阵列数据，这些场景下特征成千上万，但样本只有几十到几百个。

# 验证特征选择有效函数

In [3]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

def feature_selection(train, train_sel, target):
    clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0, n_jobs=-1)

    scores = cross_val_score(clf, train, target, cv=5)
    scores_sel = cross_val_score(clf, train_sel, target, cv=5)

    print("No Select Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    print("Features Select Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# 删除方差较小的要素（方法一）
VarianceThreshold是一种简单的基线特征选择方法。它会删除方差不符合某个阈值的所有要素。默认情况下，它会删除所有零方差要素，即在所有样本中具有相同值的要素。

In [4]:
from sklearn.feature_selection import VarianceThreshold

sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel = sel.fit(train)
train_sel = sel.transform(train)
test_sel = sel.transform(test)
print('训练数据未特征筛选维度', train.shape)
print('训练数据特征筛选维度后', train_sel.shape)
print(train_sel)

训练数据未特征筛选维度 (2000, 229)
训练数据特征筛选维度后 (2000, 25)
[[1.487e+03 6.000e+00 1.000e+00 ... 3.100e+02 3.100e+02 2.180e+02]
 [1.590e+02 5.000e+00 0.000e+00 ... 2.740e+02 2.740e+02 2.330e+02]
 [3.020e+02 5.000e+00 1.000e+00 ... 2.780e+02 2.780e+02 1.460e+02]
 ...
 [4.950e+03 4.000e+00 1.000e+00 ... 8.400e+01 8.400e+01 5.500e+01]
 [1.582e+03 3.000e+00 1.000e+00 ... 9.200e+01 9.200e+01 6.300e+01]
 [2.066e+03 4.000e+00 0.000e+00 ... 2.100e+02 2.100e+02 1.090e+02]]


In [5]:
feature_selection(train, train_sel, target)

No Select Accuracy: 0.93 (+/- 0.00)
Features Select Accuracy: 0.93 (+/- 0.00)


# 单变量特征选择（方法二）
通过基于单变量统计检验选择最佳特征。

In [6]:
from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif

sel = SelectKBest(mutual_info_classif, k=10)
sel = sel.fit(train, target)
train_sel = sel.transform(train)
test_sel = sel.transform(test)
print('训练数据未特征筛选维度', train.shape)
print('训练数据特征筛选维度后', train_sel.shape)
feature_selection(train, train_sel, target)

训练数据未特征筛选维度 (2000, 229)
训练数据特征筛选维度后 (2000, 10)
No Select Accuracy: 0.93 (+/- 0.00)
Features Select Accuracy: 0.93 (+/- 0.00)


# 递归功能消除（方法三）
选定模型拟合，进行递归拟合，每次把评分低得特征去除，重复上诉循环。

In [7]:
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=10, max_depth=2, random_state=0, n_jobs=-1)
selector = RFECV(clf, step=1, cv=2)
selector = selector.fit(train, target)
print(selector.support_)
print(selector.ranking_)

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False  True False False False  True False False False False False  True
 False False False False  True False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False Fa

# 使用模型选择特征（方法四）
## 使用LR拟合的参数进行变量选择、
（L1或L2范数进行特征选择）
LR模型采用拟合参数形式进行变量选择，筛选对回归目标影响大的

In [9]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import Normalizer

normalizer = Normalizer()
normalizer = normalizer.fit(train)

train_norm = normalizer.transform(train)
test_norm = normalizer.transform(test)

LR = LogisticRegression(penalty='l2',C=5)
LR = LR.fit(train_norm, target)
model = SelectFromModel(LR, prefit=True)
train_sel = model.transform(train)
test_sel = model.transform(test)
print('训练数据未特征筛选维度', train.shape)
print('训练数据特征筛选维度后', train_sel.shape)

训练数据未特征筛选维度 (2000, 229)
训练数据特征筛选维度后 (2000, 19)


In [10]:
feature_selection(train, train_sel, target)

No Select Accuracy: 0.93 (+/- 0.00)
Features Select Accuracy: 0.93 (+/- 0.00)


## 基于树模型特征选择
树模型基于分裂评价标准所计算的总的评分作为依据进行相关排序，然后进行特征筛选

In [11]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(train, target)

model = SelectFromModel(clf, prefit=True)
train_sel = model.transform(train)
test_sel = model.transform(test)
print('训练数据未特征筛选维度', train.shape)
print('训练数据特征筛选维度后', train_sel.shape)

训练数据未特征筛选维度 (2000, 229)
训练数据特征筛选维度后 (2000, 68)


In [12]:
df_features_import = pd.DataFrame()
df_features_import['features_import'] = clf.feature_importances_
df_features_import['features_name'] = features_columns
df_features_import.sort_values(['features_import'],ascending=0).head(30)

,features_import,features_name
0,0.085219,merchant_id
228,0.079468,xgb_clf
227,0.074071,lgb_clf
18,0.019578,seller_most_1_cnt
12,0.018210,time_stamp_std
20,0.018146,brand_most_1_cnt
21,0.017891,action_type_1_cnt
14,0.016927,seller_most_1
22,0.016911,user_cnt_0
4,0.016872,seller_nunique


In [13]:
feature_selection(train, train_sel, target)

No Select Accuracy: 0.93 (+/- 0.00)
Features Select Accuracy: 0.93 (+/- 0.00)


PCA降维（不是特征选择，是特征提取）

In [8]:

from sklearn.decomposition import PCA

pca = PCA(n_components=0.99)
pca.fit(train)
print(pca.explained_variance_ratio_)
print(pca.explained_variance_)
print(pca.n_components_)

[0.53952808 0.23390601 0.18186468 0.02745562 0.01639371]
[5602339.37043684 2428827.83875234 1888442.28695997  285093.00097763
  170228.58453105]
5
